<a href="https://colab.research.google.com/github/ijusplab/utilidades-colab/blob/master/git_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Git_Utils

## Instruções:
Para clonar um repositório, primeiro copie o url no GitHub e insira no campo `remote`.

O formato deve ser `https://github.com/<nome_da_organizacao>/<nome_do_projeto>.git`.

Em seguida, verifique se os campos `git_config_path` e `projects_path` correspondem aos caminhos no seu Drive para o arquivo de configuração do git e para a pasta de projetos.

Por fim, execute a célula.

**Atenção: o arquivo de configuração do git deve ter ao menos três linhas, nesta ordem: 1) e-mail; 2) usuário; 3) token de acesso pessoal.**

**Para instruções sobre como obter um token de acesso pessoal no GitHub, veja [este guia oficial](https://help.github.com/pt/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line#creating-a-token).**


In [0]:
#@title Dados de Configuração { vertical-output: true, display-mode: "form" }
remote = "" #@param {type:"string"}
git_config_path = "/LIIA-3R/CONFIG/git_config.txt" #@param {type:"string"}
projects_path = "/LIIA-3R/PROJETOS/" #@param {type:"string"}
action = "push" #@param ["commit-pull-push", "commit", "pull", "push", "clone"]
branch = "master" #@param ["master"] {allow-input: true}
commit_message = "" #@param {type:"string"}

import re
from google.colab import drive
from IPython.display import clear_output

ROOT = '/content/drive/My\ Drive'

def path_exists(path):
  output = !test -e {path} && echo 1 || echo 0
  return output[0] == '1'

def repo_exists(repo_path):
  if path_exists(repo_path):
    %cd {repo_path}
    output = !git rev-parse --is-inside-work-tree 2>/dev/null || echo 0
    return output[0] != '0'
  return False

def mount_drive():
  drive.mount('/content/drive')

def is_valid_remote(url):
  return re.match("^https:\/\/(.+\/){1,}(.+)\.git$", url)

def git_is_set():
  token = !git config user.password
  return len(token) > 0

def git_config(file_path):
  if not path_exists(file_path):
    raise Exception('Arquivo de configuração não localizado.')
  config_info = !cat {file_path}
  email = config_info[0]
  user = config_info[1]
  token = config_info[2]
  if not email or not user:
    raise Exception('Arquivo de configuração inválido.')
  !git config --global user.email {email}
  !git config --global user.name {user}
  !git config --global user.password {token}

def fix_gitignore(repo_path):
  if path_exists(repo_path + 'gitignore'):
    !mv {repo_path}gitignore {repo_path}.gitignore

def clone(url, projects_path):
  %cd {projects_path}
  !git clone {url}
  path = projects_path + re.search("([^\/]*)\.git$", url).group(1) + '/'
  fix_gitignore(path)
  %cd {path}
  %ls
  print('remote:')
  !git remote -v

def check_pwd(path):
  if len(path) > 0 and path_exists(path):
    %cd {path}
  pwd = %pwd
  output = !git rev-parse --is-inside-work-tree 2>/dev/null || echo 0
  if output[0] == '0':
    raise Exception('O diretório atual não é um repositório local válido.')

def pull(branch):
  !git pull origin {branch}

def push(branch, remote):
  token = !git config user.password
  url = 'https://' + token[0] + '@github.com' + remote.replace('https://github.com', '')
  !git push {url} {branch}

def commit(message):
  if len(message) == 0:
    message = 'Atualizado via Colab'
  !git add .
  !git commit -m '{message}'


# # #

print('iniciando...\n')

if not path_exists(ROOT):
  print('integrando Google Drive ao ambiente...')
  mount_drive()
  print()

print('reunindo parâmetros...')

if not is_valid_remote(remote):
  raise Exception('Remoto inválido.')
repo = re.search("([^\/]*)\.git$", remote).group(1)
projects_path = ROOT + projects_path
config_path = ROOT + git_config_path
repo_path = projects_path + repo + '/'

if not repo_exists(repo_path):
  if action != 'clone':
    raise Exception('O repositório local não foi localizado. Você deve primeiro cloná-lo.')
else:
  check_pwd(repo_path)

print()

if not git_is_set():
  print('configurando o git...')
  git_config(config_path)
  print()

if action == 'clone':
  print('clonando repositório...')
  clone(remote, projects_path)
elif action == 'pull':
  print('atualizando repositório local (pull)...')
  pull(branch)
elif action == 'push':
  print('atualizando repositório remoto (push)...')
  push(branch, remote)
elif action == 'commit':
  print('iniciando commit...')
  commit(commit_message)
elif action == 'commit-pull-push':
  print('iniciando sequência...')
  commit(commit_message)
  pull(branch)
  push(branch)

else:
  raise Exception('A ação selecionada não está implementada.')
print();

print('Script concluído.')
input('Pressione ENTER para limpar o console.')
clear_output()
